In [153]:
def read(filepath):
  with open(filepath) as f:
    text = f.read()
  
  return text

In [154]:
read('example')

'Images carry valuable information and these information can be hidden in images. In our day today life Gigabytes of new images are generated and stored, building a huge and unstructured image database, much of which can be accessed through the internet. Since image plays vital role in many applications, mining image data one of the essential features in the present research field. Image data plays vital role in every aspect of the system such as in hospital for surgery, engineering for construction, web for application and so on. The other area in image mining system is the Content Based Image Retrieval (CBIR) which performs retrieval based on the similarity defined in terms of extracted features with more objectiveness. Image mining combines the area of Content Based Image Retrieval, data mining, database. Many image retrieval systems were developed in this regard. Content Based Image Retrieval is one among them which aims at searching image databases for specific images that are sim

In [155]:
import spacy
nlp = spacy.load('en_core_web_sm',disable=['parser','tagger','ner'])

In [156]:
nlp

In [157]:
nlp.max_length = 2000000

In [158]:
def separate_punc(doc):
  return [token.text.lower() for token in nlp(doc) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t1']

In [159]:
data = read('example')

In [160]:
tokens = separate_punc(data)

In [161]:
# tokens

In [162]:
print(len(tokens))

504


In [163]:
train_len = 26
text_seq = []
for i in range(train_len,len(tokens)):
  seq = tokens[i - train_len : i]
  text_seq.append(seq)

In [164]:
text_seq[1]

['carry',
 'valuable',
 'information',
 'and',
 'these',
 'information',
 'can',
 'be',
 'hidden',
 'in',
 'images',
 'in',
 'our',
 'day',
 'today',
 'life',
 'gigabytes',
 'of',
 'new',
 'images',
 'are',
 'generated',
 'and',
 'stored',
 'building',
 'a']

In [165]:
from keras.preprocessing.text import Tokenizer

In [166]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_seq)

In [167]:
seq = tokenizer.texts_to_sequences(text_seq)

In [168]:
seq[0]

[5,
 212,
 211,
 208,
 9,
 74,
 208,
 39,
 76,
 75,
 3,
 5,
 3,
 207,
 206,
 205,
 203,
 202,
 4,
 73,
 5,
 6,
 72,
 9,
 71,
 78]

In [169]:
# tokenizer.index_word

In [170]:
vocab_size = len(tokenizer.word_counts)

In [171]:
vocab_size

213

In [172]:
import numpy as np
seq = np.array(seq)

In [173]:
# len(seq)

In [174]:
x = seq[:,:-1]

In [175]:
x.shape

(478, 25)

In [176]:
y = seq[:,-1]

In [177]:
len(y)

478

In [178]:
from keras.utils import to_categorical
y = to_categorical(y,num_classes = vocab_size + 1)

In [179]:
len(y)

478

In [180]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [181]:
model = Sequential()
model.add(Embedding(vocab_size+1,25,input_length = 25))
model.add(LSTM(50,return_sequences = True))
model.add(LSTM(50))
model.add(Dense(50,activation = 'relu'))
model.add(Dense(vocab_size+1,activation = 'softmax'))
model.compile(loss='categorical_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 25, 25)            5350      
                                                                 
 lstm_8 (LSTM)               (None, 25, 50)            15200     
                                                                 
 lstm_9 (LSTM)               (None, 50)                20200     
                                                                 
 dense_8 (Dense)             (None, 50)                2550      
                                                                 
 dense_9 (Dense)             (None, 214)               10914     
                                                                 
Total params: 54,214
Trainable params: 54,214
Non-trainable params: 0
_________________________________________________________________


In [182]:
model.fit(x,y,batch_size = 10,epochs=150,verbose = 1)

Epoch 1/150
48/48 [==============================] - 6s 28ms/step - loss: 5.2757 - accuracy: 0.0544
Epoch 2/150
48/48 [==============================] - 1s 28ms/step - loss: 4.9602 - accuracy: 0.0586
Epoch 3/150
48/48 [==============================] - 1s 28ms/step - loss: 4.8366 - accuracy: 0.0711
Epoch 4/150
48/48 [==============================] - 1s 28ms/step - loss: 4.8177 - accuracy: 0.0711
Epoch 5/150
48/48 [==============================] - 2s 35ms/step - loss: 4.7625 - accuracy: 0.0711
Epoch 6/150
48/48 [==============================] - 2s 40ms/step - loss: 4.7388 - accuracy: 0.0753
Epoch 7/150
48/48 [==============================] - 1s 28ms/step - loss: 4.6925 - accuracy: 0.0732
Epoch 8/150
48/48 [==============================] - 1s 29ms/step - loss: 4.6247 - accuracy: 0.0774
Epoch 9/150
48/48 [==============================] - 1s 28ms/step - loss: 4.5847 - accuracy: 0.0753
Epoch 10/150
48/48 [==============================] - 1s 29ms/step - loss: 4.5226 - accuracy: 0.0711

In [183]:
input_text = text_seq[0]
input_text = ' '.join(input_text)
input_text

'images carry valuable information and these information can be hidden in images in our day today life gigabytes of new images are generated and stored building'

In [184]:
# encoded_text = tokenizer.texts_to_sequences([input_text])
# encoded_text

In [185]:
# from keras.utils import pad_sequences
from tensorflow.keras.utils import pad_sequences
output_text = []
for i in range(10):
  encoded_text = tokenizer.texts_to_sequences([input_text])[0]
  print(encoded_text)
  pad = pad_sequences([encoded_text],maxlen = 25,truncating = 'pre')
  pred= model.predict(pad)
  pred_ind = np.argmax(pred,axis = 1)[0]
  print(pred_ind)
  pred_word = tokenizer.index_word[pred_ind]
  input_text = input_text + ' ' + pred_word
  # input_text = input_text[i+1:26]
  output_text.append(pred_word)
output_text

[5, 212, 211, 208, 9, 74, 208, 39, 76, 75, 3, 5, 3, 207, 206, 205, 203, 202, 4, 73, 5, 6, 72, 9, 71, 78]
1/1 [==============================] - 1s 790ms/step
13
[5, 212, 211, 208, 9, 74, 208, 39, 76, 75, 3, 5, 3, 207, 206, 205, 203, 202, 4, 73, 5, 6, 72, 9, 71, 78, 13]
1/1 [==============================] - 0s 34ms/step
40
[5, 212, 211, 208, 9, 74, 208, 39, 76, 75, 3, 5, 3, 207, 206, 205, 203, 202, 4, 73, 5, 6, 72, 9, 71, 78, 13, 40]
1/1 [==============================] - 0s 43ms/step
9
[5, 212, 211, 208, 9, 74, 208, 39, 76, 75, 3, 5, 3, 207, 206, 205, 203, 202, 4, 73, 5, 6, 72, 9, 71, 78, 13, 40, 9]
1/1 [==============================] - 0s 41ms/step
79
[5, 212, 211, 208, 9, 74, 208, 39, 76, 75, 3, 5, 3, 207, 206, 205, 203, 202, 4, 73, 5, 6, 72, 9, 71, 78, 13, 40, 9, 79]
1/1 [==============================] - 0s 38ms/step
1
[5, 212, 211, 208, 9, 74, 208, 39, 76, 75, 3, 5, 3, 207, 206, 205, 203, 202, 4, 73, 5, 6, 72, 9, 71, 78, 13, 40, 9, 79, 1]
1/1 [==============================] - 0

['a',
 'huge',
 'and',
 'unstructured',
 'image',
 'database',
 'much',
 'of',
 'which',
 'can']